In [168]:
import pandas as pd

Import geocoded data and drop unwanted column

In [169]:
fatalities = pd.read_csv('fatalities_geocoded.csv')
fatalities = fatalities.drop('Unnamed: 0', axis = 1)
fatalities.head()

,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,location,latitude,longitude
0,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,"Shelton, WA",47.215094,-123.100707
1,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,"Aloha, OR",45.494284,-122.867045
2,Phyllis Ilene Jepsen,2015-10-02,shot,knife,55.0,F,W,Aloha,OR,True,other,Not fleeing,False,"Aloha, OR",45.494284,-122.867045
3,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,"Wichita, KS",37.687176,-97.330053
4,Nicholas Garner,2015-08-22,shot,NaN,26.0,M,W,Wichita,KS,False,attack,Car,False,"Wichita, KS",37.687176,-97.330053


Import population data

In [170]:
population = pd.read_csv('population.csv')

Import state names and abbreviations

In [171]:
state_name_and_abbrev = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

Rename 'State' column in order to join population dataframe with state_name_and_abbrev dataframe; drop unwanted column and rename 'Abbreviation' column to 'state'

In [172]:
state_name_and_abbrev = state_name_and_abbrev.rename({'State' : 'state'}, axis = 1)

population = pd.merge(state_name_and_abbrev, population, how = 'inner')

population = population.drop('state', axis = 1)

population = population.rename({'Abbreviation' : 'state'}, axis = 1)

Calculate average annual rate of change for population, by state, in order to calculate the average rate of change, which is used to estimate 2019 population, by state.

In [173]:
population['diff_2015_2016'] = ((population['2016'] - population['2015']) / population['2015']) * 100
population['diff_2016_2017'] = ((population['2017'] - population['2016']) / population['2016']) * 100
population['diff_2017_2018'] = ((population['2018'] - population['2017']) / population['2017']) * 100

population['avg_diff'] = (population['diff_2015_2016'] + population['diff_2016_2017'] + population['diff_2017_2018']) / 3

population['2019'] = pd.Series(population['2018'] * ((100 + population['avg_diff']) / 100))

# Convert from float to int, to be rid of decimals and scientific notation

population['2019'] = population['2019'].astype(int)

Join fatalities dataframe with population dataframe, drop unwanted columns, and rename remaining columns

In [174]:
mega_merge = pd.merge(fatalities, population, 'inner')

mega_merge = mega_merge.drop(['diff_2015_2016', 'diff_2016_2017', 'diff_2017_2018', 'avg_diff'], axis = 1)

mega_merge = mega_merge.rename({'2015' : '2015_pop',
                                '2016' : '2016_pop',
                                '2017' : '2017_pop',
                                '2018' : '2018_pop',
                                '2019' : '2019_pop_est'}, axis = 1)

Export dataframe as .csv

In [167]:
mega_merge.to_csv('fatalities_geocoded_with_pop.csv')